In [1]:
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import h5py

from allen_v1dd.client import OPhysClient
from allen_v1dd.stimulus_analysis import LocallySparseNoise
from allen_v1dd.stimulus_analysis import analysis_tools as at
at.set_stylesheet()

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
def get_h5_group(h5_file, group_names, create_if_not_exists=True, overwrite=False):
    curr_group = h5_file

    for i in range(len(group_names)):
        name = group_names[i]
        is_last = i == len(group_names) - 1
        
        if is_last and name in curr_group and overwrite:
            del curr_group[name]

        if name in curr_group:
            curr_group = curr_group[name]
        else:
            if create_if_not_exists:
                curr_group = curr_group.create_group(name)
            else:
                return None

    return curr_group

In [ ]:
client = OPhysClient("/Users/chase/Desktop/test_v1dd_data/")

In [14]:
RF_TYPES = ("on", "off")

with h5py.File("../../data_frames/lsn-events.h5", "w") as f:
    for session_id in tqdm(client.get_all_session_ids(), desc="Loading LSN analyses"):
        session = client.load_ophys_session(session_id=session_id)
        mouse, col, vol = session.get_mouse_column_volume()

        for plane in session.get_planes():
            lsn = LocallySparseNoise(session, plane, trace_type="events")
            
            for rf_type_idx, rf_type in enumerate(RF_TYPES):
                group = get_h5_group(f, [f"M{mouse}", f"{col}{vol}", f"Plane_{plane}"], overwrite=True)
                lsn.save_to_h5(group)

Loading LSN analyses:   0%|          | 0/22 [00:00<?, ?it/s]


TypeError: super(type, obj): obj must be an instance or subtype of type